# Quantum Key Distribution Workshop

Program to implement a simple Quantum Key Distribution system. Allows for the generation of a secure key using Quantum simulators and has the option of a listener present. Quantum noise can be implemented although no error correction algorithm is included. 


The code is currently incomplete and requires the Quantum Circuit logic to be written. Follow along with the workshop (or go ahead if you want to try by yourself) and complete the code to create a functioning Quantum Key Distribution system.


Camp QMIND 2021

Code created by: Spencer Hill

In [ ]:
# Import statements. Ensure that you have qiskit and numpy installed onto your computer before running this code/cell.

from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error


import numpy as np
from numpy import random

np.random.seed(seed = 0)

# Defining the Z and X basis as numerical values (0 and 1)
Z = 0
X = 1

In [ ]:
# Initialize Quantum simulator
def simulate_circuit(qubit, noise_model):
    backend = Aer.get_backend('qasm_simulator')
    qasm_sim = Aer.get_backend('qasm_simulator')
    qobj = assemble(qubit, shots=100, memory=True)
    result = qasm_sim.run(qobj, noise_model=noise_model).result()
    measured_bit = int(result.get_memory()[0])
    return measured_bit

In [ ]:
# Noise function
def get_noise(p_meas, p_gate):
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x"]) # single qubit gate error is applied to x gates
    
    return noise_model

In [ ]:
# Function to discard any bits that were measured using different basis
def trash_different(message, bases1, bases2):
    key = []
    for bit, base1, base2 in zip(message, bases1, bases2):
        if base1 == base2:
            key.append(bit)
    return key

In [ ]:
# Function used to encode the bit message in Quantum Circuits
def encode_message(bits, basis):
    message = []
    for bit, base in zip(bits, basis):
        qc = encode_qubit(bit, base)
        message.append(qc)
    return message      

In [ ]:
# Function used to measure the Quantum Bits received by Bob
def measure_bits(message, basis, noise_model):
    backend = Aer.get_backend('qasm_simulator')
    decoded_message = []
    for qubit, base in zip(message, basis):
        qubit = measure_qubit(qubit, base)
        decoded_message.append(simulate_circuit(qubit, noise_model))
    return decoded_message        

In [ ]:
# Function to simulate Quanutm Key Distribution between Alice and Bob.
# There is an optional listener Eve that can intercept the message. 
# The number of verify bits has to be less than the number of bits divided by 2.
def distribute_key(num_bits, verify_bits, listener=False, noise=0.01):
    assert(verify_bits < num_bits / 2)
    n = num_bits
    noise_model = get_noise(noise, noise)
    
    alice_bits = random.randint(2, size=n)
    alice_bases = random.randint(2, size = n)

    message = encode_message(alice_bits, alice_bases)
    
    if listener:
        eve_bases = random.randint(2, size=n)
        intercepted_message = measure_bits(message, eve_bases, noise_model)
    
    bob_bases = random.randint(2, size=n)

    decoded = measure_bits(message, bob_bases, noise_model)
    alice_key = trash_different(alice_bits, alice_bases, bob_bases)

    bob_key = trash_different(decoded, alice_bases, bob_bases)

    alice_sample = alice_key[:verify_bits]
    bob_sample = bob_key[:verify_bits]

    if alice_sample == bob_sample:
        key = alice_key[verify_bits:]
        print("Key Communicated Successfully! The Secret Key has " + str(len(key)) + " Bits:")
        string_key = ""
        for bit in key:
            string_key += str(bit)
        print(string_key)
        return key
    else:
        print("Listener Detected! Key Compromised!")

In [ ]:
# Parameters entered into the distribute_key function. You can alter these parameters to change different
# aspects of the system. 

# Number of bits initially genearted. For a desired key length of n, this value should be 2*(n+validation_length)
bit_length = 100

# Number of validation bits used
validation_length = 15

# Boolean variable for whether an adversarial listener is overseeing your channel
listener = False

# Noise expressed as a likelihood of inducing error (e.g. 0.01 is 1% of all gate operations will induce an error)
noise = 0

## Unfinished Portion

Below are two incomplete functions needed to encode the qubits with Quantum information and measure their state once received by Bob. Add code defining the gates here to complete the program. 

Syntatically, the Quantum Circuit q can be modified in the following ways:
    - q.x(/qubit number/) to add a Pauli X gate
    - q.h(/qubit number/) to add a Hadamard Gate
    - q.measure(/qubit number/, /classical bit number/) to measure the value of a qubit
    
Recall that a Hadamard Gate transforms the qubit between the X and Z basis, while the Pauli X gate switches the amplitude of the states |0> and |1> (or |+> and |->).

In [ ]:
def measure_qubit(q, base):
    if base == Z:
        # Your code goes here

        
    else: # if base == X
        # Your code goes here

        
    return q

In [ ]:
def encode_qubit(bit, base):
    q = QuantumCircuit(1,1)
    if base == Z:
        if bit == 1:
            # Your code goes here

            
    else: # if base == X
        if bit == 1:
            # Your code goes here

            
        # Your code goes here
        
        
    q.barrier()
    return q

In [ ]:
# Run this cell to test the operation of your Quantum Key Distribution system. 
key = distribute_key(bit_length, validation_length, listener=listener, noise=noise)